# Detekcja i lokalizacja
## Inicjalizacja srodowiska


In [0]:
#!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!apt-get install -y -qq software-properties-common module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 132684 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&ac

## Mapowanie dysku GDrive do Colab Local storage

In [0]:
!mkdir -p gdrive
!google-drive-ocamlfuse gdrive

import warnings
warnings.filterwarnings("ignore")

print("Done!")

Done!


## Klonowanie Mask R-CNN 
  

In [0]:
!git clone https://github.com/matterport/Mask_RCNN.git 

import os,sys

# Root directory of the project
ROOT_DIR = os.path.abspath("/content/Mask_RCNN/")
MGR_RCNN = os.path.abspath('/content/gdrive/MGR_RCNN/')

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
sys.path.append(MGR_RCNN)

%cd Mask_RCNN
!pip3 install -r requirements.txt
!python setup.py install

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.81 MiB | 11.24 MiB/s, done.
Resolving deltas: 100% (571/571), done.
/content/Mask_RCNN
     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 163kB 55.2MB/s 
running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mrcnn
copying mrcnn/model.py -> build/lib/mrcnn
copying mrcnn/parallel_model.py -> build/li

## Tworzenie srodowiska klasyfikacji
Wraz z kopiowaniem wytrenowanej sieci do katalogow lokalnych

In [0]:
!mkdir samples/znaki
#!rm /content/Mask_RCNN/samples/znaki/znaki.py
!cp /content/gdrive/MGR_RCNN/ZNAKI/znaki.py /content/Mask_RCNN/samples/znaki/znaki.py
#!ls -l /content/Mask_RCNN/samples/znaki/

rm: cannot remove '/content/Mask_RCNN/samples/znaki/znaki.py': No such file or directory
total 20
-rw-r--r-- 1 root root 20424 Nov  9 18:40 znaki.py


## Definiowanie srodowiska (zmienne srodowiskowe i biblioteki)

In [0]:
%cd /content/Mask_RCNN
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches


import skimage.io


# Root directory of the project
ROOT_DIR = os.path.abspath("/content/Mask_RCNN")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
#import mrcnn.config as config

ZNAKI_WEIGHTS_PATH = "/content/gdrive/MGR_RCNN/COCO/logs/znaki20190708T2020/mask_rcnn_znaki_0160.h5"
WEIGHTS_PATH = "/content/gdrive/MGR_RCNN/COCO/logs/znaki20190708T2020/"

/content/Mask_RCNN


Using TensorFlow backend.


## Szczegóły detekcji znaków - importy nazw i klas

In [0]:
%cd /content/Mask_RCNN/samples/znaki/
#import znaki
from mrcnn.config import Config
from mrcnn import model as modellib, utils
class InferenceConfig(Config):
    # Run detection on one image at a time
    NAME = 'darek_det'
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 43
config = InferenceConfig()
config.display()

CLASS_NAMES = open('/content/gdrive/MGR_RCNN/ZNAKI/znaki_labels').read().strip().split("\n")


/content/Mask_RCNN/samples/znaki

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                56
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE      

In [0]:
rcnn = modellib.MaskRCNN(mode='inference', model_dir=WEIGHTS_PATH, config=InferenceConfig())








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [0]:
rcnn.load_weights('/content/gdrive/MGR_RCNN/COCO/logs/znaki20190708T2020/mask_rcnn_znaki_0160.h5', by_name=True)







Re-starting from epoch 160


## Poszczególne zdjęcia (ramki filmu) są poddawane detekcji w sieci Mask R-CNN, wyniki detekcji są zapisywane w pliku yaml

In [0]:
import yaml

PIC_DIR = '/content/gdrive/dataset/test_noc/'
source = PIC_DIR
y = '/content/gdrive/dataset/test_noc/val/detection_20191106.yaml'
for root, dirs, files in os.walk(PIC_DIR):
    for filename in files:
      if filename.rpartition(".")[-1] == 'jpg':
        #print(filename)
        image = skimage.io.imread(os.path.join(PIC_DIR,filename))
        results = rcnn.detect([image], verbose=0)
        r = results[0]
        #frame = visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], CLASS_NAMES, r['scores'])
        #pic_nam = dict(fi = str(filename))
        pic_nam = {}
        yaml_entry = {}
        yaml_entry['rois']=str(r['rois'])
        yaml_entry['class_id']=str(r['class_ids'])
        yaml_entry['scores'] = str(r['scores'])
        #yaml_entry = dict(roi = (r['rois']), class_id = str(r['class_ids']),
        #                  scores = str(r['scores'])
        #)
        pic_nam[filename] = yaml_entry
        with open(y, 'a') as yaml_file:
          yaml.dump(pic_nam, yaml_file, default_flow_style=False)

        print(filename)
#f.close()

000466.jpg
000468.jpg
000464.jpg
000467.jpg
000469.jpg
000471.jpg
000477.jpg
000474.jpg
000473.jpg
000476.jpg
000470.jpg
000472.jpg
000478.jpg
000475.jpg
000479.jpg
000480.jpg
000483.jpg
000489.jpg
000481.jpg
000486.jpg
000485.jpg
000488.jpg
000484.jpg
000482.jpg
000487.jpg
000492.jpg
000498.jpg
000499.jpg
000495.jpg
000490.jpg
000493.jpg
000491.jpg
000497.jpg
000494.jpg
000496.jpg
000509.jpg
000502.jpg
000501.jpg
000507.jpg
000500.jpg
000504.jpg
000503.jpg
000505.jpg
000506.jpg
000508.jpg
000510.jpg
000515.jpg
000519.jpg
000511.jpg
000513.jpg
000516.jpg
000518.jpg
000512.jpg
000517.jpg
000514.jpg
000525.jpg
000524.jpg
000521.jpg
000528.jpg
000522.jpg
000523.jpg
000529.jpg
000527.jpg
000520.jpg
000526.jpg
000537.jpg
000532.jpg
000530.jpg
000531.jpg
000534.jpg
000535.jpg
000539.jpg
000538.jpg
000536.jpg
000533.jpg
000540.jpg
000544.jpg
000541.jpg
000543.jpg
000549.jpg
000546.jpg
000542.jpg
000545.jpg
000547.jpg
000548.jpg
000550.jpg
000552.jpg
000554.jpg
000551.jpg
000553.jpg
000556.jpg

FileNotFoundError: ignored

# Lokalizacja geograficzna znaków i uzupełnianie brakujących danych

## Weryfikacja odleglosci / obliczenia pomocnicze

In [0]:
from geographiclib.geodesic import Geodesic
import math

geod = Geodesic.WGS84  # define the WGS84 ellipsoid


g = geod.Inverse(54.2696533, 18.589817, 54.2696838, 18.5898151)
print("The distance is {:.3f} m.".format(g['s12']))

l = geod.InverseLine(54.2696533, 18.589817, 54.2696838, 18.5898151)
ds = 3.397/2; n = int(math.ceil(l.s13 / ds))
for i in range(n + 1):
   if i == 0:
     print("distance latitude longitude azimuth")
   s = min(ds * i, l.s13)
   g = l.Position(s, Geodesic.STANDARD | Geodesic.LONG_UNROLL)
   print("{:.0f} {:.7f} {:.7f} {:.7f}".format(
     g['s12'], g['lat2'], g['lon2'], g['azi2']))

The distance is 3.397 m.
distance latitude longitude azimuth
0 54.2696533 18.5898170 -2.0882005
2 54.2696685 18.5898161 -2.0882012
3 54.2696838 18.5898151 -2.0882020
3 54.2696838 18.5898151 -2.0882020


## Wczytanie pliku z danymi gps dla każdego ze zdjęć i wykrytymi znakami
metadata.yaml - współrzędne GPS
detection_xxx.yaml - wyniki detekcji/ klasyfikacji

In [0]:
import yaml

%cd '/content/gdrive/dataset/test_noc/'
!pwd

#'/content/gdrive/dataset/test_noc/val/detection_20191106.yaml'
with open("metadata.yaml",'r') as stream:
  data = yaml.load(stream)


with open('/content/gdrive/dataset/test_noc/val/detection_20191106.yaml','r') as stream2:
  detection = yaml.load(stream2)
#data['000100.jpg']['gps']
#print('sec')
detection['000550.jpg']

/content/gdrive/dataset/test_noc
/content/gdrive/dataset/test_noc


{'class_id': '[14]',
 'rois': '[[ 378 1096  543 1248]]',
 'scores': '[0.9744985]'}

In [0]:
print(data.keys())

dict_keys(['000000.jpg', '000001.jpg', '000002.jpg', '000003.jpg', '000004.jpg', '000005.jpg', '000006.jpg', '000007.jpg', '000008.jpg', '000009.jpg', '000010.jpg', '000011.jpg', '000012.jpg', '000013.jpg', '000014.jpg', '000015.jpg', '000016.jpg', '000017.jpg', '000018.jpg', '000019.jpg', '000020.jpg', '000021.jpg', '000022.jpg', '000023.jpg', '000024.jpg', '000025.jpg', '000026.jpg', '000027.jpg', '000028.jpg', '000029.jpg', '000030.jpg', '000031.jpg', '000032.jpg', '000033.jpg', '000034.jpg', '000035.jpg', '000036.jpg', '000037.jpg', '000038.jpg', '000039.jpg', '000040.jpg', '000041.jpg', '000042.jpg', '000043.jpg', '000044.jpg', '000045.jpg', '000046.jpg', '000047.jpg', '000048.jpg', '000049.jpg', '000050.jpg', '000051.jpg', '000052.jpg', '000053.jpg', '000054.jpg', '000055.jpg', '000056.jpg', '000057.jpg', '000058.jpg', '000059.jpg', '000060.jpg', '000061.jpg', '000062.jpg', '000063.jpg', '000064.jpg', '000065.jpg', '000066.jpg', '000067.jpg', '000068.jpg', '000069.jpg', '000070.j

## Aproksymacja brakujących ramek 

In [0]:
import pandas as pd
df = pd.DataFrame({'long': 1.,
                    'lat': 1.,
                    'cat': 'foo',
                    'dist': 1.,
                    'ROIS': 'foos',
                   'scores': 'foo'}, index=data.keys()) 

In [0]:
for name in data.keys():
  #print(data[i]['gps'])
  df.at[name,'long'] = data[name]['gps']['long']
  df.at[name,'lat'] = data[name]['gps']['lat']
  df.at[name,'scores'] = detection[name]['scores']
  df.at[name,'cat'] = detection[name]['class_id']
  df.at[name,'ROIS'] = detection[name]['rois']



## Detekcja brakujacych ramek (1-3 ramek)

In [0]:
df['pop_cat3'] = df['cat'].shift(-3)
df['pop_cat2'] = df['cat'].shift(-2)
df['pop_cat'] = df['cat'].shift(-1)

df['next_cat'] = df['cat'].shift(1)
df['next_cat2'] = df['cat'].shift(2)
df['next_cat3'] = df['cat'].shift(3)
df['next_cat4'] = df['cat'].shift(4)

In [0]:
df.loc[df.pop_cat == df.next_cat, 'cat_det'] = df.pop_cat
df.loc[df['cat'] != '[]', 'cat_det'] = df.cat

In [0]:
ds = df.loc[(df['cat_det'] == '[]') & (df['pop_cat3'] != '[]') & (df['pop_cat2'] == '[]')].copy()
ds = ds.drop(['long','lat','pop_cat3'],axis=1)
df1 = df.copy()
df1['long'] = df1['long'].shift(-3)
df1['lat'] = df1['lat'].shift(-3)

## Wynik detekcji

In [0]:
result = pd.concat([ds, df1], axis=1, join='inner')
result2 = result.drop(['cat','dist','ROIS','scores','pop_cat2','pop_cat','next_cat','next_cat2','next_cat3','next_cat4','cat_det'], axis=1)
result2

,long,lat,pop_cat3
000082.jpg,18.600716,54.269230,[13]
000338.jpg,18.562090,54.346829,[13]
000493.jpg,18.562090,54.346829,[39]
000544.jpg,18.562090,54.346829,[14]
000556.jpg,NaN,NaN,NaN


## Stworz Video ze zdjec / pomocnicze 

In [0]:
def make_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
 
    The function relies on http://opencv-python-tutroals.readthedocs.org/en/latest/.
    By default, the video will have the size of the first image.
    It will resize every image to this size before adding them to the video.
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

import glob
import os


images = list(glob.iglob(os.path.join(TEMP_IMG_DIR, '*.*')))
# Sort the images by integer index
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

outvid = '/content/gdrive/dataset/video_out/out.mp4'
make_video(outvid, images, fps=30)

## Download

In [0]:
from google.colab import files
files.download('/content/gdrive/dataset/video_out/out.mp4')